In [1]:
import sys
sys.path.append(r'/home/tomas/Workspace/geo-adjust')
sys.path.append(r'/home/tomas/Workspace/geo-utils')
sys.path.append(r'/home/tomas/Workspace/forgetthebuble2.0/')

In [2]:
import sympy as sp
sp.init_printing()

from numpy import array, eye, pi, mean, diag, hstack

from geo_adjust.config import GHModelConfig
from geo_adjust.gh import GHAdjust

from dev.ipy_utils import show_matrix

from calibration.scheme import get_real_scheme
from scripts.calibrate_imu_2 import simulate_obs

# Circle adjustment GH

First create a GH Model **Config**:

In [3]:
gh_config = GHModelConfig()

Symbols for observations and parameters:

In [4]:
# observations (l)
y, x = sp.symbols('y x')
# parameters (x)
xm, ym, r = sp.symbols('xm ym r')

The circle model itself of $\varphi (l, x) = 0$:

In [5]:
# the model
phi = sp.Matrix([sp.sqrt((x - xm) ** 2 + (y - ym) ** 2) - r])

In [6]:
phi

⎡        _______________________⎤
⎢       ╱         2           2 ⎥
⎣-r + ╲╱  (x - xm)  + (y - ym)  ⎦

Let the toolbox do the rest of the work:

In [7]:
gh_config.add_model_autodiff(phi, (ym, xm, r), (y, x))

The config is ready, let's create a **Solver**:

In [8]:
solver = GHAdjust(gh_config)

Initial parameters...

In [9]:
# add initial approximations
solver.set_initial_params([120., 85., 40])

...and observation data.

In [10]:
points = array([[164.595, 73.414],
                [159.396, 62.563],
                [136.455, 45.842],
                [112.648, 46.136],
                [85.822, 71.995],
                [84.701, 95.772],
                [89.246, 106.901],
                [113.501, 125.639],
                [137.304, 125.374],
                [164.847, 97.226]])

# add data
solver.add_data(points)

Observation data must be of shape `nx2` since two observations occur in one equation.
The added data can be accesed through the solver:

In [43]:
solver._l_obs.shape

Solving the optimization problem is easy and returns the **Result** as an object of type `GHResult`:

In [12]:
result = solver.solve()
type(result)

geo_adjust.result.GHResult

All the interesting data can now be accessed through this result object:

In [13]:
print('Parameters: ', result.x)
print('Parameter std dev: ', result.sigma_x)

Parameters:  [124.97106051  85.74919574  41.50283075]
Parameter std dev:  [0.00582104 0.00634012 0.00423214]


More sophisticated information can be accessed through the properties 
* `model`
* `parameters`
* `observations`

In [14]:
result.model.print_info()


 Adjustment Model 
 - Adjustment Model: 		  GH  
              (Gauss-Helmert-Model)
 - Observations ( 2): 	  y, x
    └ Total #:   20

 - Parameters   ( 3): 	  ym, xm, r

 - Redundancy: 				     7
    ├ # of Parameters: 		     3
    ├ # of Observations: 	    20
    └ # of Equations: 		    10

------------------------------------------------------
 - Model:
    ├ Auto Differentiation:	  [ OK]
    └ Phi(x,l) = 0:
        ⎡        _______________________⎤
        ⎢       ╱         2           2 ⎥
        ⎣-r + ╲╱  (x - xm)  + (y - ym)  ⎦





In [15]:
result.model.print_summary()


 Result Summary 
 - Convergence: 				  [ OK]
    ├ # Iterations: 			     5
    └ Criterium: 					  1.000e-12
 - Hauptprobe: 				  [ OK]
    └ Max Eps: 					  1.421e-14
 - Globaltest: 				  [NOK]
    ├ Test: 						  eq 
    ├ Var Prio: 					  1.000
    ├ Var Post: 					  0.000
    ├ Ratio:    					  0.000
    ├ Alpha: 						  0.100
    └ Testquantity: 				  0.310 < 0.000 < 2.010
 - Residuals:
    ├ SSE (vTv):                0.001
    ├ MSE:                      0.000
    ├ RMSE:                     0.008
    ├ WSSE (vTPv):              0.001
    ├ WMSE:                     0.000
    └ WRMSE:                    0.000




One can see that the combatibility test (Globaltest) fails.
This is because we did not supply any stochastic model apriori.

Let's check the observations covariance information a posteriori:

In [16]:
show_matrix(result.observations.vcm)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,6.04313e-05,3.6911e-05,3.68585e-05,-2.48253e-05,8.39096e-06,-2.9159e-05,-3.99739e-06,-1.28499e-05,8.35987e-06,2.93706e-06,1.23326e-05,-3.06947e-06,9.38728e-06,-5.55795e-06,-1.06225e-06,3.69421e-06,6.15988e-06,1.97912e-05,4.21385e-05,1.2128e-05
1,3.6911e-05,0.000167509,-1.14743e-05,7.72828e-06,-2.61216e-06,9.07738e-06,1.24442e-06,4.00024e-06,-2.60248e-06,-9.14327e-07,-3.83923e-06,9.55545e-07,-2.92232e-06,1.73023e-06,3.30684e-07,-1.15003e-06,-1.91761e-06,-6.16114e-06,-1.3118e-05,-3.77553e-06
2,3.68585e-05,-1.14743e-05,8.87545e-05,6.07825e-05,9.35655e-06,-3.25144e-05,-6.4972e-06,-2.08856e-05,2.07049e-07,7.27423e-08,8.52186e-06,-2.121e-06,8.81977e-06,-5.22194e-06,1.23382e-06,-4.2909e-06,2.22779e-06,7.15773e-06,2.95166e-05,8.49526e-06
3,-2.48253e-05,7.72828e-06,6.07825e-05,0.000138061,-6.30191e-06,2.18994e-05,4.37605e-06,1.40671e-05,-1.39454e-07,-4.89941e-08,-5.73972e-06,1.42856e-06,-5.94037e-06,3.51713e-06,-8.31015e-07,2.89005e-06,-1.50048e-06,-4.82094e-06,-1.98803e-05,-5.7218e-06
4,8.39096e-06,-2.61216e-06,9.35655e-06,-6.30191e-06,0.000169386,3.34081e-05,-4.17908e-06,-1.34339e-05,-6.72108e-06,-2.36131e-06,-1.30069e-06,3.23728e-07,1.06234e-06,-6.28981e-07,1.42381e-06,-4.95162e-06,-1.33223e-06,-4.28036e-06,2.91312e-06,8.38433e-07
5,-2.9159e-05,9.07738e-06,-3.25144e-05,2.18994e-05,3.34081e-05,6.29048e-05,1.45225e-05,4.66833e-05,2.33561e-05,8.20566e-06,4.51995e-06,-1.12497e-06,-3.69168e-06,2.18574e-06,-4.94779e-06,1.72071e-05,4.62956e-06,1.48744e-05,-1.01232e-05,-2.91359e-06
6,-3.99739e-06,1.24442e-06,-6.4972e-06,4.37605e-06,-4.17908e-06,1.45225e-05,0.000168461,-3.3876e-05,1.26139e-05,4.43163e-06,6.57134e-06,-1.63554e-06,2.68446e-06,-1.5894e-06,-1.34878e-06,4.6907e-06,2.2172e-06,7.1237e-06,2.47386e-06,7.12008e-07
7,-1.28499e-05,4.00024e-06,-2.08856e-05,1.40671e-05,-1.34339e-05,4.66833e-05,-3.3876e-05,7.01028e-05,4.05481e-05,1.42457e-05,2.11239e-05,-5.25753e-06,8.62936e-06,-5.1092e-06,-4.33573e-06,1.50785e-05,7.12732e-06,2.28995e-05,7.95236e-06,2.28879e-06
8,8.35987e-06,-2.60248e-06,2.07049e-07,-1.39454e-07,-6.72108e-06,2.33561e-05,1.26139e-05,4.05481e-05,6.89999e-05,-3.8646e-05,4.19744e-05,-1.0447e-05,3.0228e-05,-1.78972e-05,2.41245e-06,-8.38984e-06,2.80169e-06,9.00163e-06,1.81232e-05,5.21608e-06
9,2.93706e-06,-9.14327e-07,7.27423e-08,-4.89941e-08,-2.36131e-06,8.20566e-06,4.43163e-06,1.42457e-05,-3.8646e-05,0.000165422,1.47468e-05,-3.67033e-06,1.062e-05,-6.28779e-06,8.47562e-07,-2.94759e-06,9.84316e-07,3.16253e-06,6.36719e-06,1.83256e-06


Or only the standard deviations:

In [17]:
result.observations.std

array([[0.00777376, 0.01294251],
       [0.00942096, 0.01174992],
       [0.01301482, 0.00793126],
       [0.01297925, 0.00837274],
       [0.00830662, 0.01286164],
       [0.00754148, 0.01309329],
       [0.00907749, 0.01204727],
       [0.01302217, 0.00806459],
       [0.01298822, 0.00852164],
       [0.00809835, 0.01302322]])

Let's also add point names and the stochastic model apriori: 

In [18]:
solver.add_data(points, sigma_ll=eye(points.size)*0.01**2, row_names=['P{:02d}'.format(i+1) for i in range(10)])

In [19]:
result = solver.solve()

In [20]:
result.model.print_summary()


 Result Summary 
 - Convergence: 				  [ OK]
    ├ # Iterations: 			     1
    └ Criterium: 					  1.000e-12
 - Hauptprobe: 				  [ OK]
    └ Max Eps: 					  1.421e-14
 - Globaltest: 				  [ OK]
    ├ Test: 						  eq 
    ├ Var Prio: 					  1.000
    ├ Var Post: 					  1.790
    ├ Ratio:    					  1.790
    ├ Alpha: 						  0.100
    └ Testquantity: 				  0.310 < 1.790 < 2.010
 - Residuals:
    ├ SSE (vTv):                0.001
    ├ MSE:                      0.000
    ├ RMSE:                     0.008
    ├ WSSE (vTPv):             12.530
    ├ WMSE:                     1.790
    └ WRMSE:                    1.790




Also define some more information on the observations:
* Units
* Names
* Conversion
* Formats

In [21]:
result.observations.set_obs_config([y, x],
                                   ['cm'] * 2,
                                   ['x', 'y'],
                                   cxs=[lambda x: x * 1e2] * 2,
                                   formats=['.2f'] * 2)

In [22]:
result.observations.print()


 Measurements
 - Adjusted Measurements, residuals, and standard deviation (σ):
    ╒════════╤══════════╤══════════╤════════════╤════════════╤════════════╤════════════╤═══════════╤═══════════╕
    │ name   │   y [cm] │   x [cm] │   v_y [cm] │   v_x [cm] │   σ_y [cm] │   σ_x [cm] │   nv_y [] │   nv_x [] │
    ╞════════╪══════════╪══════════╪════════════╪════════════╪════════════╪════════════╪═══════════╪═══════════╡
    │ P01    │ 16459.81 │  7341.30 │       0.31 │      -0.10 │       0.78 │        1.3 │       0.3 │       0.3 │
    ├────────┼──────────┼──────────┼────────────┼────────────┼────────────┼────────────┼───────────┼───────────┤
    │ P02    │ 15939.41 │  6256.43 │      -0.19 │       0.13 │       0.94 │        1.2 │       0.2 │       0.2 │
    ├────────┼──────────┼──────────┼────────────┼────────────┼────────────┼────────────┼───────────┼───────────┤
    │ P03    │ 13644.84 │  4586.49 │      -0.66 │       2.29 │       1.30 │        0.8 │       2.1 │       2.1 │
    ├────────┼──

In [23]:
result.parameters.set_params_config([ym, xm, r], ['cm']*3, ['x_m', 'y_m', 'r'],
                                   cxs=[lambda x: x * 1e2] * 3,
                                   formats=['.2f'] * 3)

In [24]:
result.parameters.print()


 Paramters
 - Parameter Estimates ± σ:
    ├   ym:     12497.11 ±     0.58   [cm]
    ├   xm:      8574.92 ±     0.63   [cm]
    └    r:      4150.28 ±     0.42   [cm]
- Correlation Matrix:
    ╒══════╤══════╤══════╕
    │ 1.00 │ 0.16 │ 0.01 │
    ├──────┼──────┼──────┤
    │ 0.16 │ 1.00 │ 0.02 │
    ├──────┼──────┼──────┤
    │ 0.01 │ 0.02 │ 1.00 │
    ╘══════╧══════╧══════╛




# Saving Models

In [25]:
gh_config.save('circle.pk')

In [26]:
%timeit gh_config.add_model_autodiff(phi, (ym, xm, r), (y, x))

10.2 ms ± 104 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [27]:
%timeit GHModelConfig.load('circle.pk')

220 µs ± 653 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [28]:
conf = GHModelConfig.load('circle.pk')

In [29]:
solver2 = GHAdjust(conf)
solver2.set_initial_params([120., 85., 40])
solver2.add_data(points, sigma_ll=eye(points.size)*0.01**2, row_names=['P{:02d}'.format(i+1) for i in range(10)])

In [30]:
result2 = solver2.solve()

In [31]:
result2.parameters.print()


 Paramters
 - Parameter Estimates ± σ:
    ├   ym:      124.971 ±    0.006
    ├   xm:       85.749 ±    0.006
    └    r:       41.503 ±    0.004
- Correlation Matrix:
    ╒══════╤══════╤══════╕
    │ 1.00 │ 0.16 │ 0.01 │
    ├──────┼──────┼──────┤
    │ 0.16 │ 1.00 │ 0.02 │
    ├──────┼──────┼──────┤
    │ 0.01 │ 0.02 │ 1.00 │
    ╘══════╧══════╧══════╛




# IMU Calibration

In [42]:
imu_config = GHModelConfig.load(r'../models/imu_rob.pk')
# sp.simplify(imu_config._sym_phi)

In [33]:
# simulate some data
scheme = get_real_scheme()

params = {'g_true': 9.82, 'g_est': 9.82,
          'psi_bi': array([[0., 0., 0.]]).T, 'l': 1.35,
          'var': 1e-5,
          'ba': array([[-0.008, -0.001, 0.016]]).T,
          'sa': array([[-400e-6, 250e-6, 100e-6]]).T + 1,
          'ma': array([[20e-3, 5e-3, -8e-3]]).T / 200 * pi,
          'mo': array([[0., 0., 0.]]).T / 200 * pi,
          'epochs': 1,
          'sets': 1,
          'sim_model': 'bias_scale_misalign_mounting',
          'estimate_model': 'bias_scale_misalign_mounting',
          'var_rob': (0.035 / 200 * pi) ** 2}

imu_obs, imu_var, rob_obs, rob_var = simulate_obs(scheme, params)

In [34]:
# compute with GH adjustment
imu_solver = GHAdjust(imu_config, max_iterations=10)

x0 = array([0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0.])
imu_solver.set_initial_params(x0)

# add the simulated data
obs_all = hstack((imu_obs, rob_obs))
var_all = hstack((imu_var, rob_var))

names = ['I{:03d}'.format(i+1) for i in range(imu_obs.shape[0])]
imu_solver.add_data(obs_all, sigma_ll=diag(var_all.flatten()), var0=mean(var_all),
                row_names=names)

In [35]:
imu_result = imu_solver.solve()

In [36]:
imu_result.model.print_summary()
imu_result.parameters.print(print_correlation=False)


 Result Summary 
 - Convergence: 				  [ OK]
    ├ # Iterations: 			     4
    └ Criterium: 					  1.000e-12
 - Hauptprobe: 				  [ OK]
    └ Max Eps: 					  1.776e-15
 - Globaltest: 				  [ OK]
    ├ Test: 						  eq 
    ├ Var Prio: 					  0.000
    ├ Var Post: 					  0.000
    ├ Ratio:    					  0.828
    ├ Alpha: 						  0.100
    └ Testquantity: 				  0.698 < 0.828 < 1.346
 - Residuals:
    ├ SSE (vTv):                0.000
    ├ MSE:                      0.000
    ├ RMSE:                     0.001
    ├ WSSE (vTPv):              0.000
    ├ WMSE:                     0.000
    └ WRMSE:                    0.000



 Paramters
 - Parameter Estimates ± σ:
    ├   bx:       -0.009 ±    0.001
    ├   by:       -0.001 ±    0.001
    ├   bz:        0.017 ±    0.001
    ├   sx:        1.000 ±    0.000
    ├   sy:        1.000 ±    0.000
    ├   sz:        1.000 ±    0.000
    ├   ax:        0.000 ±    0.000
    ├   ay:        0.000 ±    0.000
    ├   az:        0.000 ±    0.000
    ├   mx

In [37]:
x_sym, l_sym = imu_config.get_syms()
imu_result.parameters.set_params_config(tuple(x_sym),
                                    ['mm/s^2', 'mm/s^2', 'mm/s^2', 'ppm', 'ppm', 'ppm', 'mgon', 'mgon', 'mgon',
                                     'mgon', 'mgon', 'mgon'],
                                    ['bx', 'by', 'bz', 'sx', 'sy', 'sz', 'ax', 'ay', 'az', 'mx', 'my', 'mz'],
                                    cxs=[lambda x: x * 1000] * 3 + [
                                        (lambda x: (x - 1) * 1e6, lambda x: x * 1e6)] * 3 + [
                                            lambda x: x / pi * 200 * 1e3] * 6,
                                    formats=['.2f'] * 3 + ['.1f'] * 3 + ['.1f'] * 6)

imu_result.observations.set_obs_config(l_sym,
                                   ['mm/s^2'] * 3 + ['mgon'] * 2,
                                   ['f_x', 'f_y', 'f_z', 'ro', 'pi'],
                                   cxs=[lambda x: x * 1e3] * 3 + [lambda x: x / pi * 200 * 1e3] * 2,
                                   formats=['.2f'] * 3 + ['.1f'] * 3,
                                   groups=['IMU Accelerations'] * 3 + ['Rob Angles'] * 2)

In [38]:
imu_result.model.print_summary()
imu_result.parameters.print(print_correlation=False)


 Result Summary 
 - Convergence: 				  [ OK]
    ├ # Iterations: 			     4
    └ Criterium: 					  1.000e-12
 - Hauptprobe: 				  [ OK]
    └ Max Eps: 					  1.776e-15
 - Globaltest: 				  [ OK]
    ├ Test: 						  eq 
    ├ Var Prio: 					  0.000
    ├ Var Post: 					  0.000
    ├ Ratio:    					  0.828
    ├ Alpha: 						  0.100
    └ Testquantity: 				  0.698 < 0.828 < 1.346
 - Residuals:
    ├ SSE (vTv):                0.000
    ├ MSE:                      0.000
    ├ RMSE:                     0.001
    ├ WSSE (vTPv):              0.000
    ├ WMSE:                     0.000
    └ WRMSE:                    0.000



 Paramters
 - Parameter Estimates ± σ:
    ├   bx:        -8.56 ±     1.00   [mm/s^2]
    ├   by:        -1.09 ±     0.83   [mm/s^2]
    ├   bz:        16.57 ±     0.81   [mm/s^2]
    ├   sx:       -273.6 ±    190.4   [ppm]
    ├   sy:        233.6 ±    117.2   [ppm]
    ├   sz:         10.2 ±    109.4   [ppm]
    ├   ax:         26.4 ±     10.5   [mgon]
    ├   ay:     